In [1]:

from connectors.postgres_connector import PostgresConnector
from feeds.postgres_feed import PostgresFeed

import dedupe

pg_conn = PostgresConnector(db='jameycdb', username='jameyc', password='UXZSXXXSFZeU8XKw', host='silobuster-db-do-user-12298230-0.b.db.ondigitalocean.com', port=25060)
# select_qry = "SELECT id, name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
# select_qry2 = "SELECT id, name, address FROM what_location"

select_qry = "SELECT id, name, address_1, address_2, city, state_province, postal_code, url, description, duplicate_id, duplicate_type FROM organizations_normalized WHERE name IS NOT NULL AND NOT name = '' AND address_1 IS NOT NULL AND NOT address_1 = '' AND city IS NOT NULL AND NOT city = ''"
raw_def = [
    {"field": "name", "type": "String"},
    {"field": "address_1", "type": "String", "has missing": True},
    {"field": "address_2", "type": "Exists", "has missing": True},
    {"field": "city", "type": "String", "has missing": True},
    {"field": "state_province", "type": "String", "has missing": True},
    {"field": "postal_code", "type": "Exists", "has missing": True},
    {"field": "url", "type": "Exists", "has missing": True}
]


pg_feed = PostgresFeed.from_manual(connector=pg_conn, query=select_qry, column_definition=raw_def, primary_key='id')

# select_qry = "SELECT name, address FROM what_location WHERE address is not null and not address = '' and name is not null and not name = ''"
deduper = dedupe.Dedupe(pg_feed.variable_definition)



In [2]:
#deduper.prepare_training(pg_feed.formatted_data, sample_size=100000, blocked_proportion=1)
deduper.prepare_training(pg_feed.formatted_data)

In [3]:
#distinct_pairs = []
#while True:
#    try:
#        pair = deduper.uncertain_pairs()
#        distinct_pairs.extend(pair)
#    except IndexError as e:
#        break
#    except Exception as e:
#        raise

#deduper.mark_pairs({
#    "match": [],
#    "distinct": distinct_pairs
#})
#print (distinct_pairs)

In [4]:
print("Starting active labeling...")

Starting active labeling...


In [5]:
dedupe.convenience.console_label(deduper)

print ('Dumping training results...')


name : new way ministries
address_1 : 205 south british columbia avenue
address_2 : 
city : lynden
state_province : wa
postal_code : 98264
url : http://www.christianhopeassociation.org

name : new way ministries
address_1 : 205 south british columbia ave
address_2 : 
city : lynden
state_province : wa
postal_code : 98264
url : http://www.christianhopeassociation.org

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


name : whatcom family ymca
address_1 : 1256 north state str
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : http://whatcomymca.org

name : whatcom family ymca
address_1 : 1256 north state st
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : http://whatcomymca.org

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


name : salvation army - bellingham
address_1 : 1515 birchwood ave
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : https://bellingham.salvationarmy.org

name : salvation army - bellingham
address_1 : 1515 birchwood aven
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : https://bellingham.salvationarmy.org

1/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

name : sea mar community health centers
address_1 : 4455 cordata parkway
address_2 : 
city : bellingham
state_province : wa
postal_code : 98226
url : https://www.seamar.org/

name : sea mar community health center (bellingham)
address_1 : 4455 cordata pkwy
address_2 : 
city : bellingham
state_province : wa
postal_code : 98226
url : https://www.seamar.org/whatcom-medical-bellingham.html

24/10 positive, 18/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


name : whatcom county
address_1 : 311 grand ave
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : http://www.co.whatcom.wa.us

name : whatcom unified emergency coordination center
address_1 : 311 grand avenue
address_2 : 
city : bellingham
state_province : wa
postal_code : 98225
url : http://www.whatcomready.org

25/10 positive, 18/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f
Dumping training results...


Finished labeling


In [9]:
with open("training_results.txt", "w") as f:
    deduper.write_training(f)

In [7]:
#with open("training_settings.txt", "wb") as f:
#    deduper.write_settings(f)

In [12]:
#for field in deduper.fingerprinter.index_fields:
print ('training')
deduper.train() 
print('trained')

training


NoIndexError: Attempting to block with an index predicate without indexing records